# Tutorial: Beyond Linear Programming, (CPLEX Part2)

This notebook describes some special cases of LP, as well as some other non-LP techniques, and also under which conditions they should be used. 

Before continuing, you should ensure you followed the CPLEX Tutorial Part 1.

After completing this unit, you should be able to describe what a network model is, and the benefits of using network models, explain the concepts of nonlinearity and convexity, describe what a piecewise linear function is, and describe the differences between Linear Programming (LP), Integer Programming (IP), Mixed-Integer Programming (MIP),  and Quadratic Programming (QP).  You should also be able to construct a simple MIP model. 


>This notebook is part of [Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html).

>It requires a valid subscription to **Decision Optimization on Cloud** or a **local installation of CPLEX Optimizers**. 

Discover us [here](https://developer.ibm.com/docloud).

Table of contents:

* [CPLEX Modeling for Python](#Use-IBM-Decision-Optimization-CPLEX-Modeling-for-Python)
* [Network models](#Network-models)
* [Non-linearity and Convexity](#Non-linearity-and-Convexity)
* [Integer Optimization](#Integer-Optimization)
* [Quadratic Programming](#Quadratic-Programming)

We will use DOcplex to write small samples to illustrate the topics

## Use IBM Decision Optimization CPLEX Modeling for Python

Let's use the [DOcplex](https://cdn.rawgit.com/IBMDecisionOptimization/docplex-doc/2.0.15/docs/index.html) Python library to write sample models in Python.

### Step 1: Download the library

First install *docplex* if needed.

In [34]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Step 2: Set up the prescriptive engine

    Subscribe to our private cloud offer or Decision Optimization on Cloud solve service here if you do not want to use a local solver.
    Get the service URL and your personal API key and enter your credentials here if accurate:

In [35]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "api_44f5ba41-b02b-479a-bec2-3ccb0ee6927a"

# Network models

In this topic, you’ll learn what a network model is, and how its structure can be exploited for more efficient solutions.

## Networks in real life

Several problems encountered in Operations Research (OR) involve networks, such as:
Distribution problems (for example, transportation networks)
Assignment problems (for example, networks of workers and jobs they could be assigned to)
Planning problems (for example, critical path analysis for project planning)

Many network models are special LP problems for which specialized solution algorithms exist. 

It is important to know whether a problem can be formulated as a network model to exploit the special structure.

This topic introduces networks in general, as well as some well-known instances of network models.

## Network modeling concepts

Any network structure can be described using two types of objects:

- Nodes: Defined points in the network, for example warehouses.
- Arcs: An arc connects two nodes, for example a road connecting two warehouses. 

An arc can be _directed_, which means than an arc $a_{ij}$ from node $i$ to node $j$ is different from arc $a_ji$ that begins at node $j$ and ends at node $i$.

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_5.png?raw=true" >
</ul> 

 A sequence of arcs connecting two nodes is called a chain. Each arc in a chain shares exactly one node with the preceding arc.

 When all the arcs in a chain are directed such that it is possible to traverse the chain in the directions of the arcs from the start node to the end node, it is called a path.

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_6.png?raw=true" >
</ul> 

## Different types of network problems

The following are some well-known types of network problems:
- Transportation problem
- Transshipment problem
- Assignment problem
- Shortest path problem
- Critical path analysis

Next, you'll learn how to recognize each of these, and how their special structure can be exploited.

### The Transportation Problem

One of the most common real-world network problems is the transportation problem.  This type of problem involves a set of supply nodes and a set of demand nodes.  The objective is to minimize the transportation cost from the supply nodes to the demand nodes, so as to satisfy the demand, and without exceeding the suppliers’ capacities.  

Such a problem can be depicted in a graph, with supply nodes, demand nodes, and connecting arcs.  The supply capacity is indicated with the supply nodes, while the demand is indicated with the demand nodes, and the transportation costs are indicated on the arcs.  

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_8.png?raw=true" >
</ul> 

The LP formulation involves one type of variable, namely x(i,j) representing the quantity transported from supply node i to demand node j.  The objective is to minimize the total transportation cost across all arcs.  The constraints are flow conservation constraints.  The first two constraints state that the outflow from each supply node should be less than or equal to the supply capacity.  The next three constraints state that the inflow into each demand node should equal the demand at that node.   The domain for the shipments on the allowable arcs is set to be greater than or equal to zero, while the shipment quantities on the disallowed arcs are set to zero.  

Even though arcs (1,4) and (2,3) do not exist in the graph, the variables are included in the slide to show the special structure of the transportation problem.  If you were to formulate such a model in practice, you’d simply exclude these variables. 

#### Formulating a simple transportation problem with DOcplex

In the next section, we formulate the problem described above using DOcplex.

#### What data for the transpotation problem?

Input ndoes are integers ranging in {1, 2}; output nodes are integers ranging from 3 to 5.

The data consists in three Python dictionaries:

- one dictionary gives capacity values for all input nodes
- one dictionary contains demands for all target nodes
- one last dictionary holds cost values for some (source, target) pair of nodes.

In [36]:
capacities = {1: 15, 2: 20}
demands = {3: 7, 4: 10, 5: 15}
costs = {(1,3): 2, (1,5):4, (2,4):5, (2,5):3}

# Python ranges will be used to iterate on source, target nodes.
source = range(1, 3) # {1, 2}
target = range(3, 6) # {3,4,5}

#### Create a model instance

In [37]:
from docplex.mp.model import Model

tm = Model(name='transportation')

#### Define the decision variables
- The continuous variable `desk` represents the production of desk telephones.
- The continuous variable `cell` represents the production of cell phones.

In [38]:
# create flow variables for each couple of nodes
# x(i,j) is the flow going out of node i to node j
x = {(i,j): tm.continuous_var(name='x_{0}_{1}'.format(i,j)) for i in source for j in target}

# each arc comes with a cost. Minimize all costed flows
tm.minimize(tm.sum(x[i,j]*costs.get((i,j), 0) for i in source for j in target))

tm.print_information()

Model: transportation
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 0
   - linear=0
 - parameters: defaults


#### Set up the constraints

- For each source node, the total outbound flow must be smaller than available quantity.
- For each target node, total inbound flow must be greater thand demand

In [39]:
# for each node, total outgoing flow must be smaller than available quantity
for i in source:
    tm.add_constraint(tm.sum(x[i,j] for j in target) <= capacities[i])
    
# for each target node, total ingoing flow must be greater thand demand
for j in target:
    tm.add_constraint(tm.sum(x[i,j] for i in source) >= demands[j])

#### Express the business objective: minimize total flow cost

Each arc has a unit cost and we want to minimize the total cost. If an arc has no entry in the dictionary, we assume a zero cost (using the `dict.get` method.

In [40]:
tm.minimize(tm.sum(x[i,j]*costs.get((i,j), 0)))

### Solve with the Decision Optimization solve service

If url and key are None, the Modeling layer will look for a local runtime, otherwise will use the credentials.

Look at the documentation for a good understanding of the various solving/generation modes.

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

In any case, `Model.solve()` returns a solution object in Python, containing the optimal values of decision variables, if the solve succeeds, or else it returns `None`.

In [41]:
tms = tm.solve(url=url, key=key)
assert tms
tms.display()

solution for: transportation
objective: 0.000
x_1_5 = 15.000
x_2_3 = 7.000
x_2_4 = 10.000


## References
* [CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* Need help with DOcplex or to report a bug? Please go [here](https://developer.ibm.com/answers/smartspace/docloud).
* Contact us at dofeedback@wwpdl.vnet.ibm.com.